Housekeeping

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.optimize import fsolve
import itertools
from astropy.table import Table

from f_detrend import detrend_fuction

sn.set_style('whitegrid')
%matplotlib inline
%load_ext autoreload
%autoreload 2

Auxiliary functions

In [2]:
# Expanded and modified version (first column now string)
def read_table(filename,keyword='ciclo',rescale=1,ncols=5,start=2,end_row='Total',
               endword=None,omit_list=['t20_21'],index_to_int=True,debug=False):
    t0 = start
    t1 = ncols+start
    results_follow = 0
    row_names = []
    interaction = 0
    f = open(filename,'r')
    for line in f:
        words = line.split()
        if debug:
            print(words)
        if len(words)>1 and words[0] == keyword:
            results_follow=1
            if endword==None:
                colnames = words[t0:t1]
            else:
                buff_v = []
                for w in words[t0:]:
                    if w==endword:
                        break
                    else:
                        buff_v.append(w)
                colnames = buff_v[:-1]
                t1 = len(buff_v)+t0-1
            table_raw = np.empty(t1-t0)
        elif len(words)>1 and words[0] == end_row:
            break
        elif len(words) == 1:
            interaction = 0
            i_count = 0
        elif len(words)>1 and results_follow==1: # and words[0] not in omit_list
            if len(words)==2 and words[1]=="|":
                var_name = words[0]
                interaction = 1
                i_count = 0
            else:
                if interaction==0:
                    row_names.append(words[0])
                else:
                    if words[1]=='|':
                        if var_name+"_"+words[0] in row_names:
                            new_name = var_name+"_"+str(int(words[0])+1)
                        else:
                            new_name = var_name+"_"+words[0]
                        row_names.append(new_name)  
                    else:
                        if var_name+"_"+words[1] in row_names:
                            new_name = var_name+"_"+str(int(words[1])+1)
                        else:
                            new_name = var_name+"_"+words[1]
                        row_names.append(new_name)  
                
                if words[2]=='|':
                    t00= t0+1
                    t11 = t1+1
                else:
                    t00 = t0
                    t11 = t1

                buff_v = []
                for w in words[t00:t11]: 
                    w = w.replace(",","")
                    buff_v.append(w)
                table_raw = np.vstack((table_raw,np.array(buff_v,dtype=float)/rescale))
    table_raw = table_raw[1:,:]
    if debug:
        print(table_raw.shape)
        print(colnames)
    if index_to_int:
        return pd.DataFrame(table_raw, columns=colnames,index=np.array(row_names,dtype=int))
    else:
        return pd.DataFrame(table_raw, columns=colnames,index=np.array(row_names))
    
    
def read_descriptive_stats(filename,keyword_data,keyword_col,ncols,rownames,start=2,debug=False):
    t0 = start
    t1 = ncols+start
    table_raw = np.empty(t1-t0)
    f = open(filename,'r')
    for line in f:
        words = line.split()
        if debug:
            print(words)
        if len(words)>1 and words[0] == keyword_data:
            buff_v = []
            for w in words[t0:t1]: 
                w = w.replace(",","")
                buff_v.append(w)
            table_raw = np.vstack((table_raw,np.array(buff_v,dtype=float)))
        elif len(words)>1 and words[0] == keyword_col:
            colnames = words[t0:t1]

    return pd.DataFrame(table_raw[1:,:], columns=colnames, index=rownames)
 
    
def file_split(filename,path,startwords=['Summary',],endword="Total",header=None,Noobs=False):
    f = open(path+filename,'r')
    parent_file = filename[:filename.find('.')]
    di = 1
    R2s = []
    writing = 0
    for line in f:
        words = line.split()
        if len(words)>0 and 'Number' in words:
            noobs = words[-1]
        if len(words)>0 and 'R2' in words:
            R2s.append(words[-1])
        if len(words)>0 and words[0] in startwords:
            sf = open(path+parent_file+'_{}.log'.format(di), "w")
            writing = 1
            if header!=None:
                sf.write(header)
        elif len(words)>0 and words[0]==endword and writing:
            sf.write(line)
            writing = 0
            sf.close()
            di+=1
        if writing:
            sf.write(line)
    if Noobs==True:
        return (noobs,R2s)
            
def normalise_table(table):
    cols = table.columns
    table['Total'] = table.sum(axis=1).copy()
    for col in cols:
        table[col] = table[col]/table['Total']
    return table   

In [100]:
dates = []
for year in range(1987,2023):
    for quarter in range(1,5):
        dates.append(str(year)+"Q"+str(quarter))
dates = dates[1:]

t0 = 59
t05 = 71
T = len(dates)
date_dict = dict(zip(range(t0,T+t0),dates))
date_dict_reverse = dict(zip(dates,range(0,T)))
date_dict_reverse_ciclo = dict(zip(dates,range(t0,T+t0)))

time_periods_thin = []
for y in range(2006,2022):
    time_periods_thin.append(str(y))
    
time_periods = ["t08_11", "t11_14", "t14_17", "t17_20", "t20","t21"]
period_labels = ['2008-2011','2011-2014','2014-2017','2017-2020','2020','2021']
    
def rename_interaction(tab,int_subs):
    tab_chunk_1 = tab.iloc[0:7].copy()
    tab_chunk_2 = tab.iloc[7:].copy()
    tab_chunk_1.rename(index=int_subs,inplace=True)
    tab = pd.concat((tab_chunk_1,tab_chunk_2))
    return tab

In [172]:
sex = ['women','men']
states=['Permanent','Non-participant']
mx = pd.MultiIndex.from_product((states,sex))

names = ['perm','inac']
labs = ['w','m']
All_tabs = {}

for name in names:
    for lab in labs:
        coefs_par_a3040_p10 = []
        std_par_a3040_p10 = []

        obs = []
        cc = 130
        f = open('./regtabs/prob_{}_stocks_{}_age3040_parent_10_agefix_cohab.log'.format(name,lab),'r')
        for line in f:
            words = line.split()
        #     print(words)
            if len(words)>1 and 'Number' in words:
        #         print(words)
                obs.append(int(words[-1].replace(',','')))
            if len(words)>0 and 'parent_10'==words[0]:
                coefs_par_a3040_p10.append(np.round(float(words[2]),4))
                std_par_a3040_p10.append("("+str(np.round(float(words[3]),4))+")")

        bounds_par_a3040_p10 = np.array(bounds_par_a3040_p10)
        std_par_a3040_p10 = np.array(std_par_a3040_p10)
        obs = np.array(obs)

        tab = np.round(pd.DataFrame(np.vstack((coefs_par_a3040_p10,std_par_a3040_p10,obs)),columns=dates[t05:],
                                    index=[r'$\beta_1$',"std",'N']),4)

        All_tabs[name+"_"+lab] = tab

In [173]:
export_tab = pd.concat(All_tabs.values(),axis=0,keys=mx).T
export_tab_final = export_tab.loc['2018Q1':,:].copy()
export_tab_final.head()

Permanent                                           Non-participant  \
           women                       men                           women   
       $\beta_1$       std     N $\beta_1$       std     N       $\beta_1$   
2018Q1    0.9715    (0.08)  4959    1.1289  (0.0946)  4615          1.2699   
2018Q2    1.0039   (0.085)  4900    1.2691  (0.1057)  4599           1.724   
2018Q3    0.9793  (0.0816)  4874    1.2925  (0.1079)  4543          1.4256   
2018Q4    0.9514  (0.0786)  5042    1.1127  (0.0923)  4591          2.1991   
2019Q1    1.0758  (0.0872)  5074    1.1564  (0.0983)  4589          1.6624   

                                                  
                             men                  
             std     N $\beta_1$       std     N  
2018Q1   (0.227)  4922    0.7293  (0.2231)  4429  
2018Q2  (0.3724)  4804    0.8774  (0.2706)  4599  
2018Q3  (0.2741)  4771    0.8012  (0.2523)  4412  
2018Q4   (0.459)  5042    0.5748  (0.1776)  4592  
2019Q1  (0.3139)  5074    0.5717  (0.1699)  4363

In [176]:
export_tab_final.to_latex('./tables/Prob_all.tex',escape=False,
                          column_format='l|ccc|ccc|ccc|ccc',multicolumn='c')

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_9976\3581822973.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  export_tab_final.to_latex('./tables/Prob_all.tex',escape=False,


In [ ]:
from astropy.table import Table

all_tabs = []

all_names = ['mothers0k','fathers0k']
formal_names = [ "Women ","Men "]

for name in all_names:
    tab = Table.read('./regtabs/tex/prob_perm_stocks_w_age3040_parent_10_agefix_cohab.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]

Btab.to_latex('./tables/qtregs_3040_agefix_0k.tex',escape=False)

In [125]:
Table.read('./regtabs/tex/prob_perm_stocks_m_age3040_parent_10_agefix_cohab.tex').to_pandas()

InconsistentTableError: Lines in LaTeX table have to end with \\

In [119]:
full_tab = pd.read_csv('./regtabs/tex/prob_perm_stocks_w_age3040_parent_10_agefix_cohab.csv',index_col=0)
full_tab.rename(columns=dict(zip(full_tab.keys(),[int(k.replace('="',"").replace(')"',"").replace('(',"")) for k in full_tab.keys()])),inplace=True)

YY = (2022-2017)*4
T = full_tab.shape[1]
full_tab = full_tab.T
full_tab = full_tab.loc[T-YY+1:,:]
full_tab = full_tab.T
full_tab

,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
"=""""",,,,,,,,,,,,,,,,,,,,
"=""""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent""","=""permanent"""
"=""permanent""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","=""""","="""""
"=""0.act""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1"""
"=""""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)"""
"=""1.act""","=""8.651***""","=""11.32***""","=""7.952***""","=""6.484***""","=""7.853***""","=""12.60***""","=""8.108***""","=""6.171***""","=""5.597***""","=""4.663***""","=""8.763***""","=""5.133***""","=""13.72***""","=""4.574***""","=""5.411***""","=""2.691*""","=""3.416**""","=""3.012**""","=""3.536**""","=""3.478**"""
"=""""","=""(3.145)""","=""(4.162)""","=""(2.733)""","=""(2.163)""","=""(2.578)""","=""(4.125)""","=""(2.752)""","=""(2.208)""","=""(2.056)""","=""(1.763)""","=""(3.521)""","=""(1.778)""","=""(5.125)""","=""(1.734)""","=""(2.331)""","=""(1.124)""","=""(1.454)""","=""(1.171)""","=""(1.374)""","=""(1.331)"""
"=""2.act""","=""12.87***""","=""12.30***""","=""9.805***""","=""10.45***""","=""11.25***""","=""11.41***""","=""8.560***""","=""7.517***""","=""9.338***""","=""6.365***""","=""12.84***""","=""11.89***""","=""16.39***""","=""6.866***""","=""8.238***""","=""9.673***""","=""7.344***""","=""6.118***""","=""7.716***""","=""7.965***"""
"=""""","=""(5.117)""","=""(4.850)""","=""(3.666)""","=""(3.953)""","=""(4.277)""","=""(4.241)""","=""(3.129)""","=""(2.855)""","=""(3.720)""","=""(2.629)""","=""(5.716)""","=""(4.918)""","=""(7.468)""","=""(2.925)""","=""(3.501)""","=""(4.540)""","=""(3.525)""","=""(2.890)""","=""(3.624)""","=""(3.450)"""
"=""3.act""","=""8.811***""","=""8.632***""","=""5.795***""","=""5.698***""","=""4.472***""","=""9.484***""","=""5.379***""","=""5.272***""","=""6.148***""","=""5.651***""","=""9.257***""","=""6.071***""","=""13.64***""","=""5.999***""","=""8.806***""","=""8.048***""","=""11.75***""","=""9.930***""","=""7.765***""","=""5.151***"""


In [121]:
P_coef = int(np.where(full_tab.index=='="parent_10"')[0])
coef_tab = full_tab.iloc[P_coef:-2,:]
coef_tab.iloc[0,:] = [n.replace('="','').replace('"','') for n in coef_tab.iloc[0,:] ]
coef_tab.iloc[1,:] = [n.replace('="','').replace('"','') for n in coef_tab.iloc[1,:].values ]

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_9976\3012171688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coef_tab.iloc[0,:] = [n.replace('="','').replace('"','') for n in coef_tab.iloc[0,:] ]
C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_9976\3012171688.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coef_tab.iloc[1,:] = [n.replace('="','').replace('"','') for n in coef_tab.iloc[1,:].values ]


In [122]:
TT =len(dates)

coef_tab.rename(columns=dict(zip(coef_tab,dates[TT-YY:])),
                index=dict(zip(coef_tab.index,[n.replace('="','').replace('"','') for n in coef_tab.index ])))

,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4
"=""""",,,,,,,,,,,,,,,,,,,,
parent_10,0.967,1.001,0.970,0.945,1.066,1.041,0.975,0.926,1.000,0.908,1.087,1.009,0.944,1.140,0.874,0.731**,0.621***,0.587***,0.621***,0.797*
,(0.0794),(0.0846),(0.0806),(0.0780),(0.0862),(0.0847),(0.0813),(0.0796),(0.0883),(0.0846),(0.102),(0.0965),(0.0942),(0.122),(0.0995),(0.0848),(0.0684),(0.0655),(0.0682),(0.0853)
part_time,"=""1.361***""","=""1.131""","=""1.169""","=""1.447***""","=""1.365***""","=""1.270**""","=""1.518***""","=""1.398***""","=""1.499***""","=""1.821***""","=""1.660***""","=""1.238*""","=""1.378**""","=""1.174""","=""1.271""","=""1.268""","=""1.457**""","=""1.835***""","=""1.584***""","=""1.781***"""
,"=""(0.123)""","=""(0.100)""","=""(0.107)""","=""(0.129)""","=""(0.118)""","=""(0.111)""","=""(0.142)""","=""(0.130)""","=""(0.142)""","=""(0.195)""","=""(0.179)""","=""(0.129)""","=""(0.153)""","=""(0.135)""","=""(0.166)""","=""(0.156)""","=""(0.173)""","=""(0.239)""","=""(0.200)""","=""(0.225)"""
college,"=""1.183""","=""1.203""","=""1.285*""","=""1.203""","=""1.301**""","=""1.363**""","=""1.065""","=""1.063""","=""1.052""","=""1.053""","=""1.047""","=""0.974""","=""1.208""","=""1.221""","=""1.093""","=""1.270""","=""1.262""","=""1.018""","=""0.906""","=""1.326*"""
,"=""(0.113)""","=""(0.118)""","=""(0.128)""","=""(0.119)""","=""(0.129)""","=""(0.136)""","=""(0.108)""","=""(0.109)""","=""(0.111)""","=""(0.120)""","=""(0.123)""","=""(0.119)""","=""(0.146)""","=""(0.156)""","=""(0.147)""","=""(0.167)""","=""(0.162)""","=""(0.135)""","=""(0.119)""","=""(0.177)"""
erte,"=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1.858""","=""1.756***""","=""0.795""","=""0.462**""","=""0.412***""","=""0.609""","=""0.538""","=""0.511""","=""0.173*""","=""0.401""","=""1""","=""1"""
,"=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(.)""","=""(0.748)""","=""(0.241)""","=""(0.182)""","=""(0.127)""","=""(0.0980)""","=""(0.256)""","=""(0.374)""","=""(0.331)""","=""(0.142)""","=""(0.357)""","=""(.)""","=""(.)"""
30.age,"=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1""","=""1"""


In [89]:
coef_tab.rename(columns=dict(zip()))

,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
"=""""",,,,,,,,,,,,,,,,,
"=""parent_10""",0.945,1.066,1.041,0.975,0.926,1.000,0.908,1.087,1.009,0.944,1.140,0.874,0.731**,0.621***,0.587***,0.621***,0.797*
"=""""",(0.0780),(0.0862),(0.0847),(0.0813),(0.0796),(0.0883),(0.0846),(0.102),(0.0965),(0.0942),(0.122),(0.0995),(0.0848),(0.0684),(0.0655),(0.0682),(0.0853)


In [9]:
IoI = []
std_idx = []
for i,idx in enumerate(tab.index):
    if str(idx)!="nan" and idx not in tab.index[-2:]:        
        IoI.append(idx)
        std_idx.append(i+1)
tab_dummies_std_f=tab.iloc[std_idx,:].copy()
tab_dummies_std_f['coefs'] = IoI
tab_dummies_std_f.set_index('coefs',inplace=True)
tab_dummies_f=tab.loc[IoI,:].copy()
tab_dummies_f

,"Parents, 25th pctl","Parents, 50th pctl","Parents, 75th pctl"
col0,,,
$\beta_0$,$-0.212$,$1.516^{***}$,$6.298^{***}$
Tenure Partner,$0.246^{***}$,$0.320^{***}$,$0.118^{***}$
Tenure \ Partner$^{2}$,$-0.007^{***}$,$-0.005^{***}$,$0.004^{***}$
Part Time,$-1.873^{***}$,$-3.817^{***}$,$-4.302^{***}$
College,$0.966^{***}$,$0.460^{***}$,$-1.225^{***}$
Less than High-school,$-0.653^{***}$,$-1.727^{***}$,$-2.587^{***}$
Partner age,$0.032^{***}$,$0.061^{***}$,$0.096^{***}$
Partner self-employed,$-0.019$,$-0.020$,$0.298^{***}$
Partner College,$0.576^{***}$,$0.729^{***}$,$0.215^{***}$


In [12]:
def join_regtable(coefs, stds):
    cols= coefs.keys()
    #     print(cols)
    New_tab = coefs[cols[0]]
    New_tab = pd.concat((New_tab.rename(cols[0][18:]),stds[cols[0]].rename(cols[0][18:]+'_std')),axis=1)
    for name in cols[1:]:
        New_tab = pd.concat((New_tab,coefs[name].rename(name[18:]),stds[name].rename(name[18:]+'_std')),axis=1)
    return New_tab

In [13]:
join_regtable(tab_dummies_f.T, tab_dummies_std_f.T)

,,_std,,_std,{2}$,{2}$_std,,_std,,_std,...,}$,}$_std,,_std,,_std,,_std,,_std
"Parents, 25th pctl",$-0.212$,(0.1490),$0.246^{***}$,(0.0132),$-0.007^{***}$,(0.0009),$-1.873^{***}$,(0.0510),$0.966^{***}$,(0.0441),...,$-0.401^{***}$,(0.0490),$-0.337^{***}$,(0.0741),$-0.172^{**}$,(0.0803),$0.004$,(0.0883),$0.585^{***}$,(0.0389)
"Parents, 50th pctl",$1.516^{***}$,(0.2298),$0.320^{***}$,(0.0173),$-0.005^{***}$,(0.0010),$-3.817^{***}$,(0.0722),$0.460^{***}$,(0.0548),...,$0.102$,(0.0908),$-0.267^*$,(0.1389),$-0.120$,(0.1575),$-0.206$,(0.1413),$1.766^{***}$,(0.0586)
"Parents, 75th pctl",$6.298^{***}$,(0.2888),$0.118^{***}$,(0.0192),$0.004^{***}$,(0.0011),$-4.302^{***}$,(0.1856),$-1.225^{***}$,(0.0564),...,$-0.085$,(0.0892),$0.128$,(0.1062),$0.564^{***}$,(0.1302),$0.230$,(0.1431),$2.696^{***}$,(0.0712)


In [ ]:

all_data = [tab_dummies_f.T,tab_dummies_f_21.T,tab_dummies_f_22.T]
all_stds = [tab_dummies_std_f.T,tab_dummies_std_f_21.T,tab_dummies_std_f_22.T]

One_tab = {}
for i in range(3):
    coefs, stds = all_data[i], all_stds[i]
    cols= coefs.keys()
#     print(cols)
    New_tab = coefs[cols[0]]
    New_tab = pd.concat((New_tab.rename(cols[0][18:]),stds[cols[0]].rename(cols[0][18:]+'_std')),axis=1)
    for name in cols[1:]:
        New_tab = pd.concat((New_tab,coefs[name].rename(name[18:]),stds[name].rename(name[18:]+'_std')),axis=1)
        
    One_tab[2020+i] = New_tab.T.copy()
    
Final_tab = pd.concat((One_tab[2020]['Parents, 25th pctl'].rename("2020"),
           One_tab[2021]['Parents, 25th pctl'].rename("2021"),
          One_tab[2022]['Parents, 25th pctl'].rename("2022")),axis=1)

for ind in Final_tab.index[1::2]:
    Final_tab.rename({ind:""}, inplace=True)
Final_tab.replace({np.nan:'--'}, inplace=True)

Final_tab2 = pd.concat((One_tab[2020]['Parents, 50th pctl'].rename("2020"),
           One_tab[2021]['Parents, 50th pctl'].rename("2021"),
          One_tab[2022]['Parents, 50th pctl'].rename("2022")),axis=1)

for ind in Final_tab2.index[1::2]:
    Final_tab2.rename({ind:""}, inplace=True)
Final_tab2.replace({np.nan:'--'}, inplace=True)

Final_tab3 = pd.concat((One_tab[2020]['Parents, 75th pctl'].rename("2020"),
           One_tab[2021]['Parents, 75th pctl'].rename("2021"),
          One_tab[2022]['Parents, 75th pctl'].rename("2022")),axis=1)

for ind in Final_tab3.index[1::2]:
    Final_tab3.rename({ind:""}, inplace=True)
Final_tab3.replace({np.nan:'--'}, inplace=True)

Final_final_tab = pd.concat((Final_tab,Final_tab2,Final_tab3),axis=1,keys=['25th percentile','50th percentile','75th percentile'])
print(Final_final_tab.to_latex(escape=False))
Final_final_tab.to_latex('./tables/qtregs_ind_3040_basic_text.tex',escape=False)

In [11]:
Btab.to_latex?

In [13]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3035_{}_covid.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3035_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\2110688521.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3035_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5  \
                                                     25th pctl   
$\delta_{2020}$ x occ group 1                         $-0.246$   
                                                      (0.1809)   
$\delta_{2020}$ x occ group 2                          $0.306$   
                                                      (0.3648)   
$\delta_{2020}$ x occ group 3                    $0.687^{***}$   
...                                                        ...   
                                                      (0.2879)   
$\delta_{2022}$ x occ group 5 x female          $-0.361^{***}$   
                                                      (0.1214)   
N                                                      130,977   
Pseudo $R^2$                                            0.0353   

                                                                        \
                                             50th pctl       75th pctl   
$\delta_{2020}$ x occ group 1                 $-0.309$  $-1.645^{***}$   
                                              (0.2111)        (0.5313)   
$\delta_{2020}$ x occ group 2                 $-0.410$        $-0.781$   
                                              (0.4249)        (0.7141)   
$\delta_{2020}$ x occ group 3                  $0.282$        $-0.016$   
...                                                ...             ...   
                                              (0.5703)        (0.7177)   
$\delta_{2022}$ x occ group 5 x female  $-2.645^{***}$  $-1.681^{***}$   
                                              (0.5797)        (0.5556)   
N                                              130,977         130,977   
Pseudo $R^2$                                    0.0426          0.0291   

                                       Children younger than 10  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                          $-0.249$   
                                                       (0.3055)   
$\delta_{2020}$ x occ group 2                           $0.090$   
                                                       (0.3826)   
$\delta_{2020}$ x occ group 3                     $0.692^{***}$   
...                                                         ...   
                                                       (0.1943)   
$\delta_{2022}$ x occ group 5 x female            $-0.343^{**}$   
                                                       (0.1716)   
N                                                       136,192   
Pseudo $R^2$                                             0.0355   

                                                                        \
                                             50th pctl       75th pctl   
$\delta_{2020}$ x occ group 1                 $-0.277$  $-1.532^{***}$   
                                              (0.2264)        (0.5133)   
$\delta_{2020}$ x occ group 2                 $-0.423$        $-0.857$   
                                              (0.4635)        (0.5847)   
$\delta_{2020}$ x occ group 3                  $0.306$         $0.131$   
...                                                ...             ...   
                                              (0.7332)        (0.8030)   
$\delta_{2022}$ x occ group 5 x female  $-2.871^{***}$   $-1.112^{**}$   
                                              (0.5309)        (0.4593)   
N                                              136,192         136,192   
Pseudo $R^2$                                    0.0446          0.0306   

                                       Children younger than 15  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                          $-0.249$   
                                                       (0.1877)   
$\delta_{2020}$ x occ group 2                           $0.093$   
                                               

In [14]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3540_{}_covid.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3540_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\1293831293.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3540_all_covid.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5  \
                                                     25th pctl   
$\delta_{2020}$ x occ group 1                   $-0.998^{***}$   
                                                      (0.2864)   
$\delta_{2020}$ x occ group 2                         $-0.038$   
                                                      (0.2106)   
$\delta_{2020}$ x occ group 3                    $1.212^{***}$   
...                                                        ...   
                                                      (0.4155)   
$\delta_{2022}$ x occ group 5 x female          $-1.530^{***}$   
                                                      (0.3134)   
N                                                      250,030   
Pseudo $R^2$                                            0.0428   

                                                                        \
                                             50th pctl       75th pctl   
$\delta_{2020}$ x occ group 1           $-1.162^{***}$         $0.358$   
                                              (0.4407)        (0.2215)   
$\delta_{2020}$ x occ group 2                  $0.158$    $0.601^{**}$   
                                              (0.4078)        (0.2757)   
$\delta_{2020}$ x occ group 3            $1.131^{***}$        $-0.132$   
...                                                ...             ...   
                                              (0.9205)        (1.2962)   
$\delta_{2022}$ x occ group 5 x female  $-3.264^{***}$  $-2.267^{***}$   
                                              (0.5104)        (0.7575)   
N                                              250,030         250,030   
Pseudo $R^2$                                    0.0442          0.0233   

                                       Children younger than 10  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                    $-1.021^{***}$   
                                                       (0.2144)   
$\delta_{2020}$ x occ group 2                          $-0.067$   
                                                       (0.1952)   
$\delta_{2020}$ x occ group 3                     $1.129^{***}$   
...                                                         ...   
                                                       (0.3196)   
$\delta_{2022}$ x occ group 5 x female           $-1.162^{***}$   
                                                       (0.2947)   
N                                                       273,851   
Pseudo $R^2$                                             0.0451   

                                                                       \
                                             50th pctl      75th pctl   
$\delta_{2020}$ x occ group 1           $-1.011^{***}$      $0.432^*$   
                                              (0.3093)       (0.2454)   
$\delta_{2020}$ x occ group 2                  $0.143$  $0.673^{***}$   
                                              (0.4201)       (0.2414)   
$\delta_{2020}$ x occ group 3            $1.150^{***}$       $-0.152$   
...                                                ...            ...   
                                              (0.6893)       (1.4684)   
$\delta_{2022}$ x occ group 5 x female  $-3.099^{***}$  $-2.203^{**}$   
                                              (0.6606)       (0.9235)   
N                                              273,851        273,851   
Pseudo $R^2$                                    0.0515         0.0288   

                                       Children younger than 15  \
                                                      25th pctl   
$\delta_{2020}$ x occ group 1                    $-1.015^{***}$   
                                                       (0.3071)   
$\delta_{2020}$ x occ group 2                          $-0.069$   
                                                       (0.17

In [15]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3540_{}_controls.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3540_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\2711105681.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3540_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5                  \
                                         25th pctl       50th pctl   
$t$                                 $-0.014^{***}$  $-0.006^{***}$   
                                          (0.0011)        (0.0023)   
$t$ x female                         $0.020^{***}$   $0.043^{***}$   
                                          (0.0018)        (0.0030)   
unemployment rate x male             $0.036^{***}$   $0.025^{***}$   
                                          (0.0033)        (0.0068)   
unemployment rate x female           $0.046^{***}$  $-0.022^{***}$   
                                          (0.0042)        (0.0049)   
occ group 2                         $-0.609^{***}$   $0.248^{***}$   
                                          (0.0883)        (0.0727)   
occ group 3                         $-0.339^{***}$   $1.223^{***}$   
                                          (0.1301)        (0.0866)   
occ group 4                         $-2.464^{***}$  $-2.057^{***}$   
                                          (0.0856)        (0.0651)   
occ group 5                         $-3.942^{***}$  $-5.329^{***}$   
                                          (0.0755)        (0.0786)   
occ group 1 x female                       $0.021$    $0.247^{**}$   
                                          (0.1312)        (0.1242)   
occ group 2 x female                $-0.775^{***}$  $-0.446^{***}$   
                                          (0.0898)        (0.1359)   
occ group 3 x female                $-2.985^{***}$  $-3.710^{***}$   
                                          (0.1157)        (0.1638)   
occ group 4 x female                $-1.297^{***}$        $-0.215$   
                                          (0.0825)        (0.2030)   
occ group 5 x female                $-0.667^{***}$  $-0.997^{***}$   
                                          (0.0553)        (0.1325)   
$\beta_0$                            $4.342^{***}$   $8.504^{***}$   
                                          (0.0846)        (0.1040)   
N                                          250,030         250,030   
Pseudo $R^2$                                0.0428          0.0442   

                                           Children younger than 10  \
                                 75th pctl                25th pctl   
$t$                         $-0.013^{***}$           $-0.013^{***}$   
                                  (0.0019)                 (0.0008)   
$t$ x female                 $0.023^{***}$            $0.017^{***}$   
                                  (0.0027)                 (0.0016)   
unemployment rate x male    $-0.018^{***}$            $0.034^{***}$   
                                  (0.0063)                 (0.0025)   
unemployment rate x female  $-0.077^{***}$            $0.049^{***}$   
                                  (0.0038)                 (0.0039)   
occ group 2                  $1.535^{***}$           $-0.638^{***}$   
                                  (0.0594)                 (0.1266)   
occ group 3                  $2.557^{***}$           $-0.549^{***}$   
                                  (0.0776)                 (0.1096)   
occ group 4                  $0.513^{***}$           $-2.577^{***}$   
                                  (0.0478)                 (0.0885)   
occ group 5                 $-3.028^{***}$           $-3.981^{***}$   
                                  (0.1481)                 (0.0810)   
occ group 1 x female         $1.131^{***}$                 $-0.037$   
                                  (0.1047)                 (0.1093)   
occ group 2 x female               $0.052$           $-0.943^{***}$   
                                  (0.1256)                 (0.1298)   
occ group 3 x female        $-1.759^{***}$           $-2.988^{***}$   
                                  (0.1091)                 (0.0995)   
occ group 4 x female         $0.897^{***}$           $-1.277^{***}$   
           

In [16]:
all_tabs = []

all_names = ['5','10','15']
formal_names = [ "Children younger than 5","Children younger than 10","Children younger than 15"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_advanced_age3035_{}_controls.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_advanced_age3035_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\2287887052.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_advanced_age3035_all_controls.tex',escape=False,column_format='l|ccc|ccc|ccc',multicolumn_format='c')


Children younger than 5                  \
                                         25th pctl       50th pctl   
$t$                                 $-0.008^{***}$  $-0.009^{***}$   
                                          (0.0011)        (0.0019)   
$t$ x female                         $0.005^{***}$   $0.027^{***}$   
                                          (0.0014)        (0.0030)   
unemployment rate x male             $0.035^{***}$   $0.072^{***}$   
                                          (0.0029)        (0.0057)   
unemployment rate x female           $0.062^{***}$   $0.061^{***}$   
                                          (0.0032)        (0.0060)   
occ group 2                          $-0.218^{**}$   $0.574^{***}$   
                                          (0.0985)        (0.1160)   
occ group 3                         $-0.659^{***}$   $0.435^{***}$   
                                          (0.1004)        (0.1614)   
occ group 4                         $-1.407^{***}$  $-0.954^{***}$   
                                          (0.0655)        (0.1003)   
occ group 5                         $-2.324^{***}$  $-3.404^{***}$   
                                          (0.0621)        (0.1227)   
occ group 1 x female                      $-0.105$         $0.229$   
                                          (0.1055)        (0.1485)   
occ group 2 x female                $-0.725^{***}$  $-0.755^{***}$   
                                          (0.0905)        (0.1265)   
occ group 3 x female                $-1.347^{***}$  $-1.943^{***}$   
                                          (0.1028)        (0.1240)   
occ group 4 x female                $-0.684^{***}$         $0.051$   
                                          (0.0922)        (0.2154)   
occ group 5 x female                $-0.506^{***}$  $-0.704^{***}$   
                                          (0.0460)        (0.0771)   
$\beta_0$                            $2.428^{***}$   $4.636^{***}$   
                                          (0.0662)        (0.1192)   
N                                          130,977         130,977   
Pseudo $R^2$                                0.0353          0.0426   

                                           Children younger than 10  \
                                 75th pctl                25th pctl   
$t$                          $0.018^{***}$           $-0.008^{***}$   
                                  (0.0017)                 (0.0011)   
$t$ x female                 $0.017^{***}$            $0.005^{***}$   
                                  (0.0024)                 (0.0012)   
unemployment rate x male     $0.021^{***}$            $0.035^{***}$   
                                  (0.0061)                 (0.0028)   
unemployment rate x female   $0.015^{***}$            $0.061^{***}$   
                                  (0.0050)                 (0.0029)   
occ group 2                  $1.469^{***}$            $-0.208^{**}$   
                                  (0.0901)                 (0.0951)   
occ group 3                  $1.779^{***}$           $-0.668^{***}$   
                                  (0.1294)                 (0.0946)   
occ group 4                  $1.170^{***}$           $-1.430^{***}$   
                                  (0.0810)                 (0.0516)   
occ group 5                 $-2.087^{***}$           $-2.323^{***}$   
                                  (0.1926)                 (0.0682)   
occ group 1 x female          $0.204^{**}$                 $-0.078$   
                                  (0.0999)                 (0.1070)   
occ group 2 x female        $-1.002^{***}$           $-0.769^{***}$   
                                  (0.0777)                 (0.0933)   
occ group 3 x female        $-1.479^{***}$           $-1.387^{***}$   
                                  (0.1299)                 (0.0969)   
occ group 4 x female            $-0.237^*$           $-0.730^{***}$   
           

In [20]:
all_tabs = []

all_names = ['mothers_age3040_10','mothers0k_age3040']
formal_names = [ "Mothers","Women without Children"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_figure6_{}_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/sqtreg_figure6_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\1840227655.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/sqtreg_figure6_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')


Mothers                  \
                                             25th pctl       50th pctl   
Tenure Partner                           $0.098^{***}$   $0.191^{***}$   
                                              (0.0019)        (0.0027)   
$t$                                      $0.035^{***}$   $0.047^{***}$   
                                              (0.0038)        (0.0057)   
$t^2$                                   $-0.001^{***}$  $-0.001^{***}$   
                                              (0.0001)        (0.0001)   
$t$ x female                             $0.038^{***}$        $-0.011$   
                                              (0.0054)        (0.0081)   
$t^2$ x female                          $-0.000^{***}$   $0.001^{***}$   
                                              (0.0001)        (0.0002)   
$\delta_{2020}$                          $0.362^{***}$        $-0.180$   
                                              (0.0734)        (0.1572)   
$\delta_{2021}$                          $0.718^{***}$         $0.152$   
                                              (0.1488)        (0.2225)   
$\delta_{2022}$                          $1.113^{***}$         $0.149$   
                                              (0.1481)        (0.3122)   
female                                  $-0.855^{***}$  $-1.173^{***}$   
                                              (0.0659)        (0.0992)   
$\delta_{2020}$ x female                $-0.311^{***}$   $-0.650^{**}$   
                                              (0.0975)        (0.2757)   
$\delta_{2021}$ x female                $-0.611^{***}$  $-1.781^{***}$   
                                              (0.1442)        (0.3221)   
$\delta_{2022}$ x female                $-0.852^{***}$  $-2.220^{***}$   
                                              (0.1743)        (0.4224)   
Part time x male                        $-1.817^{***}$  $-4.008^{***}$   
                                              (0.0472)        (0.0849)   
Part time x female                      $-0.913^{***}$  $-1.389^{***}$   
                                              (0.0224)        (0.0524)   
College x male                           $1.063^{***}$   $0.563^{***}$   
                                              (0.0362)        (0.0453)   
College x female                         $1.422^{***}$   $1.737^{***}$   
                                              (0.0312)        (0.0324)   
Less than High-school x male            $-0.760^{***}$  $-1.931^{***}$   
                                              (0.0686)        (0.1088)   
Less than High-school x female          $-0.423^{***}$  $-1.479^{***}$   
                                              (0.0351)        (0.0625)   
Partner self-employed x male                   $0.010$        $-0.024$   
                                              (0.0574)        (0.0730)   
Partner self-employed x female                $-0.015$   $0.174^{***}$   
                                              (0.0315)        (0.0447)   
Partner college x male                   $0.699^{***}$   $0.867^{***}$   
                                              (0.0423)        (0.0573)   
Partner college x female                 $0.599^{***}$   $0.494^{***}$   
                                              (0.0329)        (0.0395)   
Partner less than High-school x male    $-0.255^{***}$  $-0.838^{***}$   
                                              (0.0431)        (0.0983)   
Partner less than High-school x female  $-0.392^{***}$  $-1.044^{***}$   
                                              (0.0417)        (0.0799)   
Age $>$ 35                               $0.698^{***}$   $1.893^{***}$   
                                              (0.0216)        (0.0375)   
$\beta_0$                                $0.794^{***}$   $3.444^{***}$   
                                              (0.0412)        (0.0671)   
N                                              315,

In [21]:
all_tabs = []

all_names = ['mothers_age3040_10','mothers0k_age3040']
formal_names = [ "Mothers","Women without Children"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_figure5_{}_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/ssqtreg_figure5_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\547750710.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/ssqtreg_figure5_allwomen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')


Mothers                                  \
                                    25th pctl       50th pctl       75th pctl   
$\beta_0$                       $0.640^{***}$   $3.000^{***}$   $6.403^{***}$   
                                     (0.0652)        (0.1387)        (0.1970)   
Tenure Partner                  $0.191^{***}$   $0.319^{***}$   $0.197^{***}$   
                                     (0.0056)        (0.0076)        (0.0089)   
Tenure \ Partner$^{2}$         $-0.006^{***}$  $-0.007^{***}$  $-0.002^{***}$   
                                     (0.0003)        (0.0003)        (0.0004)   
Part Time                      $-0.920^{***}$  $-1.388^{***}$  $-1.048^{***}$   
                                     (0.0203)        (0.0398)        (0.0340)   
College                         $1.378^{***}$   $1.734^{***}$    $0.074^{**}$   
                                     (0.0402)        (0.0395)        (0.0340)   
Less than High-school          $-0.398^{***}$  $-1.375^{***}$  $-2.987^{***}$   
                                     (0.0479)        (0.0789)        (0.1651)   
Partner age                    $-0.012^{***}$  $-0.019^{***}$   $0.042^{***}$   
                                     (0.0018)        (0.0039)        (0.0054)   
Partner self-employed                 $0.026$   $0.225^{***}$   $0.190^{***}$   
                                     (0.0325)        (0.0413)        (0.0416)   
Partner College                 $0.528^{***}$   $0.366^{***}$   $-0.075^{**}$   
                                     (0.0376)        (0.0348)        (0.0361)   
Partner less than High-school  $-0.364^{***}$  $-0.974^{***}$  $-1.793^{***}$   
                                     (0.0257)        (0.0741)        (0.1326)   
$\delta_{2008-2011}$            $0.245^{***}$        $-0.040$  $-0.350^{***}$   
                                     (0.0289)        (0.0577)        (0.0686)   
$\delta_{2011-2014}$            $1.016^{***}$   $0.694^{***}$   $-0.156^{**}$   
                                     (0.0349)        (0.0609)        (0.0644)   
$\delta_{2014-2017}$            $1.096^{***}$   $1.308^{***}$   $0.416^{***}$   
                                     (0.0503)        (0.0523)        (0.0434)   
$\delta_{2017-2020}$            $0.372^{***}$   $1.297^{***}$   $0.692^{***}$   
                                     (0.0455)        (0.0721)        (0.0572)   
$\delta_{2020}$                 $0.281^{***}$   $0.836^{***}$   $0.886^{***}$   
                                     (0.0567)        (0.0922)        (0.0871)   
$\delta_{2021}$                       $0.059$         $0.159$   $0.433^{***}$   
                                     (0.0685)        (0.1400)        (0.0746)   
$\delta_{2022}$                      $-0.008$   $-0.249^{**}$         $0.080$   
                                     (0.0611)        (0.1182)        (0.1355)   
Age $>$ 35                      $0.758^{***}$   $1.957^{***}$   $3.006^{***}$   
                                     (0.0181)        (0.0261)        (0.0480)   
N                                     183,530         183,530         183,530   
Pseudo $R^2$                           0.0539          0.0778          0.0753   

                              Women without Children                  \
                                           25th pctl       50th pctl   
$\beta_0$                              $0.913^{***}$   $2.971^{***}$   
                                            (0.1327)        (0.1810)   
Tenure Partner                         $0.144^{***}$   $0.271^{***}$   
                                            (0.0053)        (0.0113)   
Tenure \ Partner$^{2}$                $-0.005^{***}$  $-0.008^{***}$   
                                            (0.0002)        (0.0005)   
Part Time                             $-1.230^{***}$  $-2.434^{***}$   
                                            (0.0367)        (0.0525)   
College                                $0.429^{***}$   $0.493^{***}$   
     

In [22]:
all_tabs = []

all_names = ['fathers_age3040_10','fathers0k_age3040']
formal_names = [ "Fathers","Men without Children"]

for name in all_names:
    tab = Table.read('./regtabs/sqtreg_figure5_{}_agefix.tex'.format(name)).to_pandas()
    tab = tab.set_index('col0')
    tab.rename(columns=dict(zip(tab.keys(),[i.replace('Parents,',"") for i in tab.keys()])),inplace=True)
    all_tabs.append(tab)
    
Btab = pd.concat(all_tabs,axis=1,keys=(formal_names))

Btab.index = [str(i).replace("nan","") for i in Btab.index]
Btab.to_latex('./tables/ssqtreg_figure5_allmen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')
Btab

C:\Users\lafuentemart\AppData\Local\Temp\ipykernel_7808\4132090700.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  Btab.to_latex('./tables/ssqtreg_figure5_allmen_age3040_10_agefix.tex',escape=False,column_format='l|ccc|ccc',multicolumn_format='c')


Fathers                                  \
                                    25th pctl       50th pctl       75th pctl   
$\beta_0$                            $-0.212$   $1.516^{***}$   $6.298^{***}$   
                                     (0.1490)        (0.2298)        (0.2888)   
Tenure Partner                  $0.246^{***}$   $0.320^{***}$   $0.118^{***}$   
                                     (0.0132)        (0.0173)        (0.0192)   
Tenure \ Partner$^{2}$         $-0.007^{***}$  $-0.005^{***}$   $0.004^{***}$   
                                     (0.0009)        (0.0010)        (0.0011)   
Part Time                      $-1.873^{***}$  $-3.817^{***}$  $-4.302^{***}$   
                                     (0.0510)        (0.0722)        (0.1856)   
College                         $0.966^{***}$   $0.460^{***}$  $-1.225^{***}$   
                                     (0.0441)        (0.0548)        (0.0564)   
Less than High-school          $-0.653^{***}$  $-1.727^{***}$  $-2.587^{***}$   
                                     (0.0582)        (0.1165)        (0.2020)   
Partner age                     $0.032^{***}$   $0.061^{***}$   $0.096^{***}$   
                                     (0.0050)        (0.0080)        (0.0087)   
Partner self-employed                $-0.019$        $-0.020$   $0.298^{***}$   
                                     (0.0697)        (0.0993)        (0.0742)   
Partner College                 $0.576^{***}$   $0.729^{***}$   $0.215^{***}$   
                                     (0.0450)        (0.0666)        (0.0579)   
Partner less than High-school  $-0.190^{***}$  $-0.804^{***}$  $-1.822^{***}$   
                                     (0.0565)        (0.1414)        (0.1748)   
$\delta_{2008-2011}$                 $-0.069$         $0.041$  $-0.202^{***}$   
                                     (0.0518)        (0.0615)        (0.0729)   
$\delta_{2011-2014}$            $0.443^{***}$   $0.536^{***}$        $-0.118$   
                                     (0.0514)        (0.0718)        (0.0719)   
$\delta_{2014-2017}$            $-0.111^{**}$   $0.315^{***}$  $-0.216^{***}$   
                                     (0.0515)        (0.0764)        (0.0713)   
$\delta_{2017-2020}$           $-0.401^{***}$         $0.102$        $-0.085$   
                                     (0.0490)        (0.0908)        (0.0892)   
$\delta_{2020}$                $-0.337^{***}$      $-0.267^*$         $0.128$   
                                     (0.0741)        (0.1389)        (0.1062)   
$\delta_{2021}$                 $-0.172^{**}$        $-0.120$   $0.564^{***}$   
                                     (0.0803)        (0.1575)        (0.1302)   
$\delta_{2022}$                       $0.004$        $-0.206$         $0.230$   
                                     (0.0883)        (0.1413)        (0.1431)   
Age $>$ 35                      $0.585^{***}$   $1.766^{***}$   $2.696^{***}$   
                                     (0.0389)        (0.0586)        (0.0712)   
N                                     131,745         131,745         131,745   
Pseudo $R^2$                           0.0428          0.0619          0.0628   

                              Men without Children                  \
                                         25th pctl       50th pctl   
$\beta_0$                            $1.177^{***}$   $4.158^{***}$   
                                          (0.1157)        (0.1985)   
Tenure Partner                       $0.236^{***}$   $0.369^{***}$   
                                          (0.0078)        (0.0126)   
Tenure \ Partner$^{2}$              $-0.008^{***}$  $-0.009^{***}$   
                                          (0.0005)        (0.0013)   
Part Time                           $-1.489^{***}$  $-3.294^{***}$   
                                          (0.0425)        (0.0722)   
College                              $0.378^{***}$        $-0.111$   
                           